In [90]:
import numpy as np
import pandas as pd

from warnings import filterwarnings
filterwarnings('ignore')

In [91]:
import yfinance as yf
dat = yf.Ticker("BTC")
y = dat.history(interval='5m')['Close']
y

Datetime
2024-12-11 09:30:00-05:00    43.849998
2024-12-11 09:35:00-05:00    43.840000
2024-12-11 09:40:00-05:00    43.939899
2024-12-11 09:45:00-05:00    44.056198
2024-12-11 09:50:00-05:00    44.099998
                               ...    
2025-01-10 15:35:00-05:00    41.977001
2025-01-10 15:40:00-05:00    42.000099
2025-01-10 15:45:00-05:00    41.985001
2025-01-10 15:50:00-05:00    42.097900
2025-01-10 15:55:00-05:00    41.970001
Name: Close, Length: 1522, dtype: float64

# Klasyfikacja

Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Na razie ustawiamy klasyfikację na podstawie poprzednich 4 godzin.

In [95]:
interval = '5m' # dane 5-minutowe
window = int(60/int(interval[:-1]))
window

12

In [96]:
y = dat.history(interval=interval)['Close'].pct_change().dropna()

In [101]:
a = [1, 2, 3, 4]
a[:-1]

[1, 2, 3]

In [ ]:
# Okna zachodzące na siebie
for i in range(len(y), window, -1):
    temp_y = y.iloc[i-window:i]
    print(len(temp_y))
    print(temp_y)
    print()
    
# Okna niezachodzące na siebie
...

12
Datetime
2025-01-10 15:00:00-05:00   -0.000670
2025-01-10 15:05:00-05:00    0.000670
2025-01-10 15:10:00-05:00   -0.001420
2025-01-10 15:15:00-05:00   -0.001799
2025-01-10 15:20:00-05:00   -0.001522
2025-01-10 15:25:00-05:00    0.000951
2025-01-10 15:30:00-05:00   -0.001425
2025-01-10 15:35:00-05:00   -0.001499
2025-01-10 15:40:00-05:00    0.000550
2025-01-10 15:45:00-05:00   -0.000359
2025-01-10 15:50:00-05:00    0.002689
2025-01-10 15:55:00-05:00   -0.003038
Name: Close, dtype: float64

12
Datetime
2025-01-10 14:55:00-05:00    0.000947
2025-01-10 15:00:00-05:00   -0.000670
2025-01-10 15:05:00-05:00    0.000670
2025-01-10 15:10:00-05:00   -0.001420
2025-01-10 15:15:00-05:00   -0.001799
2025-01-10 15:20:00-05:00   -0.001522
2025-01-10 15:25:00-05:00    0.000951
2025-01-10 15:30:00-05:00   -0.001425
2025-01-10 15:35:00-05:00   -0.001499
2025-01-10 15:40:00-05:00    0.000550
2025-01-10 15:45:00-05:00   -0.000359
2025-01-10 15:50:00-05:00    0.002689
Name: Close, dtype: float64

12
Dat

In [107]:
list(range(5, 0))

[]